In [2]:
!pip install nbstripout

DEPRECATION: Loading egg at /Users/bhavanaanand/.local/lib/python3.11/site-packages/mylinear_cpp_handwritten-0.0.0-py3.11-macosx-15.5-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/bhavanaanand/.local/lib/python3.11/site-packages/mylinear_cpp_builtin-0.0.0-py3.11-macosx-15.5-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!nbstripout base_dataset_roberta.ipynb

In [4]:
!jupyter nbconvert base_dataset_roberta.ipynb \
  --to notebook \
  --execute \
  --inplace


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbconvert` not found.


In [1]:
!jupyter nbconvert --to notebook base_dataset_roberta.ipynb --output base_dataset_roberta.ipynb


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbconvert` not found.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import random
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)


In [ ]:
#seeding for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)

seed_everything(42)

In [4]:
TRAIN_JSON_PATH = "/content/drive/MyDrive/nlp/train.json"
DEV_JSON_PATH   = "/content/drive/MyDrive/nlp/dev.json"

In [5]:
def load_json_records(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return list(data.values()) if isinstance(data, dict) else data

train_records = load_json_records(TRAIN_JSON_PATH)
dev_records   = load_json_records(DEV_JSON_PATH)

In [6]:
def load_json_records(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return list(data.values()) if isinstance(data, dict) else data

train_records = load_json_records(TRAIN_JSON_PATH)
dev_records   = load_json_records(DEV_JSON_PATH)


In [ ]:
#building input examples
def build_examples(records):
    # flattening nested lists so every item is a dict
    def flatten(xs):
        for x in xs:
            if isinstance(x, list):
                yield from flatten(x)
            else:
                yield x


    if isinstance(records, dict):
        records = [records]

    records = list(flatten(records))

    out = []

    for r in records:
        if not isinstance(r, dict):
            continue

        pre = (r.get("precontext") or "").strip()
        sent = (r.get("sentence") or "").strip()
        end = (r.get("ending") or "").strip()
        meaning = (r.get("judged_meaning") or "").strip()

        meaning_txt = (
            "Meaning (intended definition):\n"
            f"{meaning}"
        )

        story_parts = [pre, sent]
        if end:
            story_parts.append(end)

        story_txt = "Story:\n" + "\n".join(story_parts)

        ex_sent = (r.get("example_sentence") or "").strip()
        if ex_sent:
            story_txt += f"\nExample sentence: {ex_sent}"

        label = float(r.get("average", 0.0))

        gid = f"{r.get('homonym', '')}||{sent}"

        out.append({
            "context": story_txt,
            "meaning": meaning_txt,
            "label": label,
            "stdev": float(r.get("stdev", 0)),
            "group_id": gid,
            "choices": r.get("choices"),
            "nonsensical": r.get("nonsensical"),
            "sample_id": r.get("sample_id")
        })

    return out


In [ ]:
#loading dataset
train_examples = build_examples(train_records)
dev_examples   = build_examples(dev_records)

train_df = pd.DataFrame(train_examples)
dev_df   = pd.DataFrame(dev_examples)

dev_labels = dev_df["label"].to_numpy(float)
dev_stdevs = dev_df["stdev"].to_numpy(float)
dev_groups = dev_df["group_id"].tolist()

In [ ]:
from sklearn.model_selection import train_test_split

# splitting original train_df into 80/20
train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    shuffle=True,
)

# building HF DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df_split, preserve_index=False),
    "validation": Dataset.from_pandas(val_df_split, preserve_index=False),
    "test": Dataset.from_pandas(dev_df, preserve_index=False)
})

In [ ]:
#model
MODEL_NAME = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(
        batch["context"],
        batch["meaning"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized = dataset.map(tokenize_batch, batched=True)
tokenized = tokenized.rename_column("label", "labels")

# Remove non-tensor columns
for col in ["context", "meaning", "stdev", "group_id", "sample_id"]:
    if col in tokenized["train"].column_names:
        tokenized = tokenized.remove_columns(col)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1824 [00:00<?, ? examples/s]

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,
    problem_type="regression"
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.clip(np.squeeze(preds), 1.0, 5.0)
    labels = np.squeeze(labels)

    spearman_corr = spearmanr(labels, preds).correlation
    mae = np.mean(np.abs(labels - preds))

    return {
        "spearman": float(spearman_corr),
        "mae": float(mae),
    }

args = TrainingArguments(
    output_dir= None,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="spearman",
    greater_is_better=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.06,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=20,
    save_total_limit=1,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"], 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2087916016.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: banand (banand-university-of-massachusetts-amherst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Spearman,Mae
1,1.544300,1.402328,0.045891,1.023489
2,1.308200,1.424862,0.045427,1.024829
3,1.326500,1.362300,0.203577,1.001480
4,1.140900,1.388653,0.314077,0.987746
5,0.834500,1.405425,0.330649,0.967405
6,0.597800,1.491716,0.350595,0.969528
7,0.453600,1.613129,0.355013,1.005506
8,0.391000,1.495106,0.378847,0.968827
9,0.255100,1.509133,0.377789,0.974947
10,0.246300,1.590987,0.381656,1.000003


TrainOutput(global_step=1140, training_loss=1.0221399470379477, metrics={'train_runtime': 96.4095, 'train_samples_per_second': 189.193, 'train_steps_per_second': 11.825, 'total_flos': 2399551280087040.0, 'train_loss': 1.0221399470379477, 'epoch': 10.0})

In [ ]:
print("\nFINAL EVAL ON DEV (TEST)")

pred = trainer.predict(tokenized["test"])
raw_preds = np.clip(np.squeeze(pred.predictions), 1.0, 5.0)

# Global Spearman & MAE 
global_spearman = spearmanr(dev_labels, raw_preds).correlation
global_mae = np.mean(np.abs(dev_labels - raw_preds))

# Macro-Spearman 
group_indices = defaultdict(list)
for i, gid in enumerate(dev_groups):
    group_indices[gid].append(i)

group_sps = []
for gid, idxs in group_indices.items():
    g_true = dev_labels[idxs]
    g_pred = raw_preds[idxs]

    if np.all(g_true == g_true[0]):
        continue

    corr = spearmanr(g_true, g_pred).correlation
    if not np.isnan(corr):
        group_sps.append(corr)

macro_spearman = float(np.mean(group_sps)) if group_sps else 0.0

# Accuracy within stdev 
errors = np.abs(raw_preds - dev_labels)
within = errors <= dev_stdevs
acc_stdev = float(np.mean(within))

print(f"Global Spearman:       {global_spearman:.4f}")
print(f"Macro Spearman:        {macro_spearman:.4f}")
print(f"MAE:                   {global_mae:.4f}")
print(f"Accuracy within stdev: {acc_stdev:.4f}")



FINAL EVAL ON DEV (TEST)


Global Spearman:       0.3817
Macro Spearman:        0.3801
MAE:                   1.0000
Accuracy within stdev: 0.5442


In [ ]:
#bootstrapping
def bootstrap_test_metrics(
    y_true,
    y_pred,
    y_stdev,
    groups,
    n_bootstrap=1000,
    seed=42
):
    rng = np.random.default_rng(seed)
    n = len(y_true)

    global_sps = []
    macro_sps = []
    maes = []
    acc_stdevs = []

    group_to_indices = defaultdict(list)
    for i, gid in enumerate(groups):
        group_to_indices[gid].append(i)

    for _ in range(n_bootstrap):
        idx = rng.integers(0, n, size=n)

        bt_true = y_true[idx]
        bt_pred = y_pred[idx]
        bt_stdev = y_stdev[idx]
        bt_groups = [groups[i] for i in idx]

        # ---- Global Spearman ----
        rho = spearmanr(bt_true, bt_pred).correlation
        global_sps.append(rho)

        # ---- MAE ----
        maes.append(np.mean(np.abs(bt_true - bt_pred)))

        # ---- Accuracy within stdev ----
        acc_stdevs.append(np.mean(np.abs(bt_true - bt_pred) <= bt_stdev))

        # ---- Macro Spearman ----
        local_group_map = defaultdict(list)
        for i, g in enumerate(bt_groups):
            local_group_map[g].append(i)

        group_corrs = []
        for g, idxs in local_group_map.items():
            gt = bt_true[idxs]
            gp = bt_pred[idxs]

            if np.all(gt == gt[0]):
                continue

            c = spearmanr(gt, gp).correlation
            if not np.isnan(c):
                group_corrs.append(c)

        if group_corrs:
            macro_sps.append(np.mean(group_corrs))
        else:
            macro_sps.append(np.nan)

    def summarize(arr):
        arr = np.array(arr, dtype=float)
        return {
            "mean": float(np.nanmean(arr)),
            "ci_low": float(np.nanpercentile(arr, 2.5)),
            "ci_high": float(np.nanpercentile(arr, 97.5)),
        }

    return {
        "global_spearman": summarize(global_sps),
        "macro_spearman": summarize(macro_sps),
        "mae": summarize(maes),
        "acc_within_stdev": summarize(acc_stdevs),
    }


In [16]:
print("\nBOOTSTRAP RESULTS (TEST SET)")

bootstrap_results = bootstrap_test_metrics(
    y_true=dev_labels,
    y_pred=raw_preds,
    y_stdev=dev_stdevs,
    groups=dev_groups,
    n_bootstrap=1000
)

for metric, stats in bootstrap_results.items():
    print(
        f"{metric:20s}: "
        f"{stats['mean']:.4f} "
        f"[{stats['ci_low']:.4f}, {stats['ci_high']:.4f}]"
    )


BOOTSTRAP RESULTS (TEST SET)


/tmp/ipython-input-3268535183.py:52: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  c = spearmanr(gt, gp).correlation


global_spearman     : 0.3811 [0.3100, 0.4476]
macro_spearman      : 0.3462 [0.2512, 0.4370]
mae                 : 1.0002 [0.9424, 1.0649]
acc_within_stdev    : 0.5443 [0.5068, 0.5834]
